In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter1d

In [ ]:
import tightbinding2d as tb

In [ ]:
# read data
data_path = "/tmp/results"
datafile = data_path + "/eigensystem.npz"
data = np.load(datafile)
energies = data['eigenvalues']
states = data['eigenvectors']
Lx, Ly = int(data['Lx']), int(data['Ly'])

In [ ]:
n_states = energies.size
sites = np.asarray(tb.lattice_sites(Lx, Ly))

In [ ]:
fig, ax_dos, edges, hist = tb.histogram(energies, n_states // 20)
ax_dos.set_title("DoS, [{:.3f}, {:.3f}]".format(energies[0], energies[-1]))
plt.show()

In [ ]:
centers = 0.5 * (edges[:-1] + edges[1:])
dos_smooth = gaussian_filter1d(hist, sigma=1.5)
# plt.plot(centers, dos_smooth, marker='o', markersize=3, linewidth=0.2, color='orangered')
plt.plot(centers, dos_smooth, linewidth=0.5, color='orangered')

In [ ]:
xs, ys = sites[:, 0], sites[:, 1]
T = 1e-4
STM_weight = lambda e, E0, T: 1 - np.tanh((0.5 / T) * (e - E0))**2

Vs = np.hstack((np.linspace(energies[0], 1.4, 500), np.linspace(3 - 0.25, 3 + 0.25, 500)))

s_2 = states[:, :]**2
slice_ = np.s_[::2]

for i_E, E_ in enumerate(Vs):
    ws = STM_weight(energies, E_, T)
    ldos_avg = np.mean(s_2 * ws, axis=1)
    # ldos_smooth = tb.gaussianSmoothing(sites, ldos_avg, sigma=1.0)
    ldos_ = ldos_avg
    vmin, vmax = np.amin(ldos_), np.amax(ldos_)
    ldos_ = (ldos_ - vmin) * (1 / (vmax - vmin))
    title = "LDoS at E = {:.3f}".format(E_)
    fig, ax1 = plt.subplots(nrows=1, ncols=1)
    scatter = ax1.scatter(xs[slice_], ys[slice_], c=ldos_[slice_], marker='s', s=35, cmap='inferno')
    ax1.set_title(title)
    cbar = fig.colorbar(scatter)
    plt.savefig("/tmp/results/ldos_avg_{:d}.png".format(i_E))
    plt.show()

In [ ]:
dEs = energies[1:] - energies[:-1]
dE_min = np.amin(dEs)
dE_max = np.amax(dEs)
print("dE: min = {:.3e}, max = {:.3e}".format(dE_min, dE_max))

fig, ax_dE, edges, hist  = tb.histogram(dEs, dEs.size // 10)
ax_dE.set_title("dE")
plt.show()

In [ ]:
dE_est = 2e-3
T = 10 * dE_est

In [ ]:
n_avg = 5
sites_a = sites[::2]
xs, ys = sites_a[:, 0], sites_a[:, 1]
T = 1e-4
dE = 5e-3
STM_weight_box = lambda e, dV, T: nF(-(e - dE), T) * nF(e + dE, T)

for i_s1 in range(0, n_states // 10):
    i_begin = i_s1 - n_avg 
    if i_begin < 0:
        i_begin = 0

    i_end = i_s1 + n_avg 
    if i_end >= n_states:
        i_end = -1

    ldos_avg = np.mean(states[::2, i_begin:i_end]**2, axis=1)
    ldos_smooth = tb.gaussianSmoothing(sites_a, ldos_avg, sigma=2.0)
    title = "states in energy range [{:.3f}, {:.3f}]".format(energies[i_s1], energies[i_s2])
    fig, ax1 = plt.subplots(nrows=1, ncols=1)
    scatter = ax1.scatter(xs, ys, c=ldos_smooth, marker='s', s=40, cmap='inferno')
    ax1.set_title(title)
    plt.savefig("/tmp/results/ldos_avg_{:d}.png".format(i_s1))
    plt.show()

In [ ]:
T = 1e-4
nF = lambda e, T: 0.5 * (1 - np.tanh(0.5 / T * e))
STM_weight_box = lambda e, E0, dE, T: nF(-(e - (E0 - dE)), T) * nF(e - (E0 + dE), T)

i_s = 10
E_ = energies[i_s]
E_ = 0.9
dE = 0.1
ws = STM_weight_box(energies, E_, dE, T)

print(f"E = {E_:.3f}")
plt.plot(energies, ws)

In [ ]:
T = 1e-4  # temperature
STM_weight = lambda e, dV, T: 1.0 - np.tanh((0.5 / T) * (e - dV))**2

nF = lambda e, T: 0.5 * (1 - np.tanh(0.5 / T * e))
w0 = 2e-3
STM_weight_box = lambda e, dV, T: nF(-(e - (dV - w0)), T) * nF(e - (dV + w0), T)

fig, ax, *_ = tb.histogram(energies, n_states // 20)
ax.set_title("DoS, [{:.3f}, {:.3f}]".format(energies[0], energies[-1]))

n_Vpts = 1 << 3
dVs = np.linspace(np.amin(energies), np.amax(energies), n_Vpts)
for dV in dVs:
    ws = STM_weight_box(energies, dV, T)
    ax.plot(energies, ws, linewidth=0, marker='x', markersize=1, label=f"dV = {dV:.3f}")
    
plt.legend()

In [ ]:
for i_e, E_i in enumerate(energies):
    print(f"{i_e:d}) {E_i:.5f}")

In [ ]:
dVs

In [ ]:
dV = dVs[1]
ws = STM_weight_box(energies, dV, T)

for i_e, E_i in enumerate(energies):
    print(f"{i_e:d}) {E_i:.5f}: weight = {ws[i_e]:.3e}")

In [ ]:
xs, ys = sites[:, 0], sites[:, 1]
n_Vpts = 1 << 3
dVs = np.linspace(np.amin(energies), np.amax(energies), n_Vpts)

T = 1e-3
w0 = 1e-5
nF = lambda e, T: 0.5 * (1 - np.tanh(0.5 / T * e))
STM_weight_box = lambda e, dV, T: nF(-(e - (dV - w0)), T) * nF(e - (dV + w0), T)

for dV in dVs:
    ldos_avg = np.mean(states**2 * STM_weight_box(energies, dV, T), axis=1)
    ldos_smooth = tb.gaussianSmoothing(sites, ldos_avg, sigma=2.0)
    title = "dI/dV at V = {:.3f}".format(dV)
    fig, ax1 = plt.subplots(nrows=1, ncols=1)
    scatter = ax1.scatter(xs, ys, c=ldos_smooth, s=10, cmap='inferno')
    cbar = fig.colorbar(scatter)
    ax1.set_title(title)
    plt.show()

In [ ]:
dV = 0.6036182515258072
STM_weight = lambda e, w, dV, T: nF(e, T) * nF(-(e-w), T)

ws = STM_weight(energies, dV, T=0.01)

s_2 = states**2
i_s1, i_s2 = 10, 15
ldos_avg = np.mean(s_2[:, i_s1:i_s2+1], axis=1)
ldos_smooth = tb.gaussianSmoothing(sites, ldos_avg, sigma=2)
plt.scatter(xs, ys, c=ldos_smooth, s=10, cmap='inferno')
plt.show()
plt.plot(energies, ws, marker='x', markersize=2, linewidth=0)
plt.show()

In [ ]:
STM_weight = lambda e, dV, T: 1.0 - np.tanh((0.5 / T) * (e - dV))**2
ws = STM_weight(energies, 0.8, T=0.9)
plt.plot(energies, ws)

In [ ]:
i_s1, i_s2 = 2167, 2186
i_s1, i_s2 = 2167, i_s1 + 10

s_2 = states**2

ldos_avg = np.mean(s_2[:, i_s1:i_s2], axis=1)
ldos_smooth = tb.gaussianSmoothing(sites, ldos_avg, sigma=2)
plt.scatter(xs, ys, c=ldos_smooth, s=10, cmap='inferno')
plt.show()
